<h1> Pick and Place Example

In [1]:
#===============================================
#Remove when moved to rmstudio
import sys
sys.path.append('/home/nvidia/dev_rmstudio/lib') 
#===============================================

#Imports
import numpy as np
import open3d  
import rmlib

rm = rmlib.RMLib()

Robot Ready


### <u>Lets start by defining a waypoint. There are two ways of saving a waypoint: 
1. The first way is to save the pose of the tool-tip (TCP) which is stored as an array of [X, Y, Z, rX, rY, rZ]. 
<br>
2. The second way is to store the joint angles of the robot which is also stored as and array with [J1, J2, J3, J4, J5, J6] with Ji is the angle at joint i in radians for a 6-DoF robot.

In this example we will store our waypoints as joint angles because it eliminates the possibility of overrotating a joint. 
<br><br>
We will also save the camera transform. The camera transform is a transformation matrix describing the position and orientation of the camera with respect to the base of the robot. We will use the camera transform later to transform objects from a local frame (relative to the camera) to a global frame (relative to the robot base.)
<br>
#### <b>Position the robot in an orientation that it can see the objects

In [28]:
# Normalize the wrist joint to give us the largest joint freedom when picking
rm.align_gripper_with_axis()
pickup_waypoint = rm.get_joint_angles()

camera_pose = rm.get_base_to_camera_pose()

### <u>Next lets tune the disparity shift
The disparity shift is a camera setting that corrects for the parallax of the two camera lenses. The disparity shift is dependent on the mean height of the point cloud. If the disparity is not set correctly your cloud will be distorted. This particular tuning method sets the disparity based on the estimated distance to the surface. We only need to tune this once because we will be coming back to the same height for each object.

In [7]:
disp = rm.tune_disparity_shift()

Disparity shif can also be set maualy if needed.

In [29]:
disp = rm.set_disparity_shift(100)

### <u>Lets define our drop-off point
#### <b>Move the robot to the desired drop point

In [30]:
rm.align_gripper_with_axis()
drop_waypoint = rm.get_joint_angles()

### <u> Now lets find some objects!
Lets start by getting our point cloud and making some modifications:

In [31]:
# Move to our initial waypoint.
rm.set_joint_angles(pickup_waypoint) 
# Retrieve point cloud.
cloud = rm.get_point_cloud()
# Compress the point cloud with a voxel size of 3mm.
cloud_vg = rm.downsample_cloud(cloud,leaf_size=0.003)
# Remove the table.
cloud_vg_nt = rm.remove_planar_surface(cloud_vg)

Now lets segment objects pick the highest object, and generate a pose to pick it up.

In [32]:
view = rm.PC_Viewer()
view.add_cloud(cloud_vg)
view.add_cloud(cloud_vg_nt,True)
view.show()

Renderer(camera=PerspectiveCamera(aspect=1.6, fov=90.0, position=(-0.0061044835976149575, 0.33397638005764607,…

In [33]:
# Segment objects with spreading segmentation algorithm.
object_clouds = rm.segment_cloud_dbscan(cloud_vg_nt,search_radius=0.0033)
# Filter out eroneous objects
object_clouds = rm.filter_clouds_size(object_clouds,dim_1=[0,100],dim_2=[0,100],dim_3=[0,100],cluster_size=[50,1000000])[1]
if len(object_clouds) > 0:
    print('Objects found: {}'.format(len(object_clouds)))
    # Sort object clouds by height.
    object_clouds_sorted = rm.sort_clouds_height(object_clouds)
    # Pick the highest cloud as our object.
    my_object = object_clouds_sorted[0]
    # Get the width and pose of our object
    object_width = rm.get_cloud_width(my_object)
    object_pose = rm.get_cloud_pose(my_object)
    global_pose = rm.align_pose_to_tcp(camera_pose.dot(object_pose))
    # Create a poses for grasping our object
    grasp_pose = rm.shift_pose_to_grasp(rm.transform_points(cloud_vg,camera_pose),global_pose,object_width)
    grasp_pose_above = grasp_pose.copy()
    grasp_pose_above[2,3] = grasp_pose_above[2,3] + 0.1
    
else:
    print("No objects found!")

Objects found: 1


### <u>Lets view the point cloud to make sure we like the proposed grasp

In [34]:
# Initialize a viewer object
view = rm.PC_Viewer()
view.add_cloud(rm.transform_points(cloud_vg,camera_pose),colorize=True,color=[100,100,100])
view.add_cloud(rm.transform_points(cloud_vg_nt,camera_pose))
for object_cloud in object_clouds:
    view.add_cloud(rm.transform_points(object_cloud,camera_pose),colorize=True)
view.add_axis(global_pose)

# We can also generate boxes representing the gripper given a transform and gripper width
gripper_boxes,finger_boxes = rm.get_gripper_boxes(grasp_pose,object_width)
view.add_gripper_boxes(gripper_boxes,finger_boxes)
view.show(view='base')

Renderer(camera=PerspectiveCamera(aspect=1.6, fov=90.0, position=(0.049136369046155044, -0.41474705789046695, …

### <u>Now lets pick up the object and place it

In [35]:
import time
rm.set_gripper_width(object_width*1.5)

# Set the torque low in case the gripper collides with something
rm.set_gripper_torque(15)

rm.movej(grasp_pose_above)
rm.movel(grasp_pose)

rm.set_gripper_torque(200)
time.sleep(0.5)
rm.close_gripper()

rm.movel(grasp_pose_above)
rm.set_joint_angles(drop_waypoint)

rm.open_gripper()

1